In [1]:
import numpy as np
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import tscv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [22]:
x = np.random.rand(10000000, 10)
y = np.random.rand(10000000)

In [23]:
# Define custom preprocessor
class StandardScaler:
    def fit(self, x):
        self.mean_ = np.mean(x, axis=0)
        self.std_ = np.std(x, axis=0)

    def transform(self, x):
        return (x - self.mean_) / self.std_
    
    def fit_transform(self, x):
        self.fit(x)
        return self.transform(x)

In [24]:
# Define custom model
class CustomLinearModel:
    def fit(self, x, y):
        self.coef_ = np.linalg.pinv(x) @ y

    def predict(self, x):
        return x @ self.coef_

In [25]:
# Define custom metrics
class MeanSquaredError:
    def evaluate(self, y_obs, y_pred):
        return np.mean((y_obs - y_pred) ** 2)
    
class MeanAbsoluteError:
    def evaluate(self, y_obs, y_pred):
        return np.mean(np.abs(y_obs - y_pred))

In [41]:
# Set up cross-validation
cross_validator = tscv.TimeSeriesCrossValidator(
    train_starts=[0, 20000, 40000],
    train_sizes=[100000, 100000, 100000],
    test_sizes=[20000, 20000, 20000],
    preprocessor=StandardScaler(),
    max_workers=1
)

In [36]:
# Define model and preprocessor
model = CustomLinearModel()

In [37]:
# Define metrics
metrics = [MeanSquaredError(), MeanAbsoluteError()]

In [42]:
# Evaluate model
evaluations = cross_validator.cross_validate(model, x, y, metrics)
print("Evaluation Results:")
for evaluation in evaluations:
    print(evaluation)

Evaluation Results:
{'MeanSquaredError': 0.33304955355444277, 'MeanAbsoluteError': 0.4989618235172669}
{'MeanSquaredError': 0.3364138574219597, 'MeanAbsoluteError': 0.5025800786638265}
{'MeanSquaredError': 0.3381258633309923, 'MeanAbsoluteError': 0.5035340007375838}
